In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./data/checkins.csv.gz', compression='gzip', index_col='id')
print(data.shape)
data.head()

(396634, 5)


,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [3]:
from sklearn.cluster import MeanShift

cluster = MeanShift(bandwidth=0.1, n_jobs=-1)
cluster.fit(data[['latitude', 'longitude']].head(100000))

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=-1, seeds=None)

In [4]:
from itertools import groupby
from operator import itemgetter

big_cluster_labels = [i for i, k in groupby(sorted(cluster.labels_)) if len(list(k)) > 15]

In [5]:
big_cluster_centers = [cluster.cluster_centers_[label] for label in big_cluster_labels]
for c in big_cluster_centers[0:20]:
    print("{},{}".format(c[0], c[1]))

40.71771639727507,-73.9918354198967
33.44943805020126,-112.00213969017547
33.44638027037988,-111.90188756212359
41.87824377967115,-87.62984336226329
37.68868157406161,-122.40933037359147
38.8861652155993,-77.04878333074303
33.357344562325096,-111.82265410760392
33.76663623218336,-84.39328918481657
42.363218639848895,-71.07368760857386
47.60624471741767,-122.33204382627093
36.117229142990276,-115.17107342280688
34.06039755458241,-118.24870902659876
44.97794782033687,-93.2673008852605
30.26718361698159,-97.74311928133027
40.76687624004166,-73.83335349045205
39.735830152625304,-104.98658042770822
39.95168037300773,-75.16273592391683
34.03548695312116,-118.43899771946148
32.98089338217789,-117.07811797821928
32.80302053531547,-96.76989743494408


In [6]:
offices = [
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134]
]

In [7]:
from geopy.distance import vincenty

def dist(a, b):
    return vincenty(tuple(a), tuple(b)).meters

In [11]:
distances = []

for office in offices:
    for center in big_cluster_centers:
        pair = (dist(center, office), tuple(center))
        distances.append(pair)

distances = sorted(distances, key=lambda el: el[0], reverse=False)

In [12]:
distances[0:20]

[(823.4439405652802, (52.37296399032261, 4.8923172225806466)),
 (868.754889081817, (-33.860630428571433, 151.20477592857145)),
 (2499.8389367887366, (25.8456722642857, -80.318890596428574)),
 (3475.8459456398728, (51.502991260887086, -0.12553728870967767)),
 (7473.232758058908, (33.809877955263097, -118.14892380690813)),
 (13989.508525635409, (25.785812419967499, -80.217938036825402)),
 (18446.965858161107, (25.705349721052581, -80.28342873815798)),
 (19965.422193191756, (33.888325342758598, -118.04892817172427)),
 (20083.481829298064, (26.010098249285683, -80.199990585714318)),
 (20972.68871913518, (33.872986011570177, -118.36209114655645)),
 (24618.591403758826, (33.97257482142858, -118.16837066666663)),
 (28509.664310873613, (33.81730643390889, -117.89124917095801)),
 (30052.960808513977, (26.138843786842077, -80.334346836842073)),
 (30781.408060008325, (33.983935874038437, -118.00740497307689)),
 (31753.935186627205, (33.674302659765758, -117.85878926777275)),
 (32558.263281921383,

In [13]:
# решение задания
print(distances[0][1])

(52.37296399032261, 4.8923172225806466)
